## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


## Import dataset

In [ ]:
SWDATA = pd.read_csv('DATAPOINT.csv')
pd.crosstab(index= SWDATA ["class"],columns="count")
SWDATA2= SWDATA.drop(["Label","Vy ","Dy ","Dm ","Vu","Du "], axis=1)
SWDATA3= SWDATA2.dropna()
pd.crosstab(index= SWDATA3["class"],columns="count")
SWDATA3.describe()


## Correlation matrix

In [ ]:
corr_matrix = SWDATA3.corr()
corr_matrix2 = np.round(corr_matrix, 2)
plt.subplots(figsize=(16, 16))
sns.heatmap(corr_matrix2, annot=True, cmap='plasma')
plt.yticks(rotation=0)
plt.show()


## Box plot

In [ ]:
for column in SWDATA3:
    plt.figure()
    sns.boxplot(x= SWDATA3[column])


## Train and Test split

In [ ]:
SSS1 = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=50)
stratified_split = []
for train_i, test_i in SSS1.split(SWDATA3, SWDATA3["class"]):
  stratified_training_set_n = SWDATA3.iloc[train_i]
  stratified_testing_set_n = SWDATA3.iloc[test_i]
  stratified_split.append([stratified_train_set_n, stratified_test_set_n])

for i, split in enumerate(stratified_split):
    training_set, testing_set = split
    print(f"Training set {i+1}:")
    print(training_set)

training_set
testing_set

testing_set["class"].value_counts()/len(testing_set)
training_set["class"].value_counts()/len(training_set)
training_set = training_set.drop("class", axis=1)
testing_set = testing_set.drop("class", axis=1)
training_features = training_set.drop("Vm ", axis=1)
training_target = training_set["Vm "]
testing_features = testing_set.drop("Vm ", axis=1)
testing_target = testing_set["Vm "]
testing_target.to_frame()



## Scaling and transformation

In [ ]:
from sklearn.compose import TransformedTargetRegressor
MMS = MinMaxScaler()
training_values = MMS.fit_transform(training_features)
training_values
Model=TransformedTargetRegressor(RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2,random_state=50),                                  transformer=MMS)
model.fit(training_values,  training_target.to_frame())



## Training prediction

In [ ]:
Pred_model = model.predict(training_values)
Pred_model
training_target_df = training_target.to_frame() if not isinstance(training_target, pd.DataFrame) else training_target
pred_model_df = pd.DataFrame(Pred_model, columns=['Predicted'])
results = pd.concat([training_target_df.reset_index(drop=True), pred_model_df], axis=1)
print(results)
r2 = r2_score(training_target, Pred_model)
average_r2 = np.mean(r2_scores)
print("Average R^2 Score:", average_r2)
MAE = mean_absolute_error(training_target, Pred_model)
R2 = r2_score(training_target, Pred_model)
RMSE = mean_squared_error(training_target, Pred_model, squared=False)
print("MAE: {:.4f}".format(MAE),"R2: {:.4f}".format(R2),"RMSE: {:.4f}".format(RMSE))
labels = ['RMSE', 'MAE', 'R2']
values = [RMSE, MAE, R2]



## Testing prediction

In [ ]:
testing_values = MMS.transform(testing_features)
testing_values
Pred_model_for_testing = model.predict(testing_values)
Pred_model_for_testing
testing_target_df = testing_target.to_frame() if not isinstance(testing_target, pd.DataFrame) else testing_target
pred_model_for_testing_df = pd.DataFrame(Pred_model_for_testing, columns=['Predicted'])
results = pd.concat([testing_target_df.reset_index(drop=True), pred_model_for_testing_df], axis=1)
print(results)

MAE = mean_absolute_error(testing_target, Pred_model_for_testing)
R2 = r2_score(testing_target, Pred_model_for_testing)
RMSE = mean_squared_error(testing_target, Pred_model_for_testing, squared=False)
print("MAE: {:.4f}".format(MAE),
      "R2: {:.4f}".format(R2),"RMSE: {:.4f}".format(RMSE))

print("MAE: {:.4f}".format(MAE),"R2: {:.4f}".format(R2),"RMSE: {:.4f}".format(RMSE))
x_values = range(len(testing_target))


## Graph (actual Vs. predicted)

In [ ]:
plt.plot(x_values, testing_target, color='black', label='Actual', linewidth=1)
plt.plot(Pred_model_for_testing, color = 'red', label = 'Predicted', linestyle = '-.', linewidth = 2)
plt.grid(alpha = 0.3)
plt.xlabel('Experiment')
plt.ylabel('Vm (kN)')
plt.legend()
plt.show()

plt.figure(figsize=(5,5))
plt.scatter(testing_target, Pred_model_for_testing,label='Testing set',edgecolor='orange', c= 'none')
plt.scatter(training_target, Pred_model,label='Training set', c='none',edgecolor='green')

testing_target_flat = np.ravel(testing_target)
training_target_flat = np.ravel(training_target)
Pred_model_for_testing_flat = np.ravel(Pred_model_for_testing)

A1 = max(max(Pred_model_for_testing_flat), max(training_target_flat))
A2 = min(min(Pred_model_for_testing_flat), min(training_target_flat))
plt.plot([A1, A2], [A1, A2], 'b--')
plt.xlabel('Measured shear strength (kN)', fontsize=10)
plt.ylabel('Predicted shear strength (kN)', fontsize=10)
plt.axis('equal')
plt.legend()
plt.show()


## Cross-validation



In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, training_values, training_target, cv=12, scoring='neg_mean_squared_error')
scores1 = cross_val_score(model, training_values, training_target, cv=12, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(model, training_values, training_target, cv=12, scoring='r2')

print("Cross-validation Mean MSE: {:.4f}".format(-scores.mean()))
print("Cross-validation Mean MAE: {:.4f}".format(-scores1.mean()))
print("Cross-validation R2: {:.4f}".format(scores2.mean()))


## Feature selection

In [ ]:
from sklearn.feature_selection import RFECV
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2,random_state=50)
selector = RFECV(estimator=rf_regressor, step=1, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)
selector.fit(training_values, training_target)
print("No. of selected features:", selector.n_features_)
print("Selected features:", selector.support_)



## Feature importance

In [ ]:
rf_regressor.fit(training_values, training_target)
training_values_df = pd.DataFrame(training_values)

COL= training_values_df.columns
importances = rf_regressor.feature_importances_
COL = training_values_df.columns
Feature_importnace = pd.DataFrame(importances, index = COL, columns = ['Importance']).sort_values(by = 'Importance', ascending = False )

sns.barplot(x= Feature_importnace.index, y='Importance', data=importance_df, color= "yellow")
plt.xlabel('Feature',fontsize=18)
plt.ylabel('Importance',fontsize=18)
plt.title('Random Forest',fontsize=20)
plt.ylim(0,0.5)
plt.show()


## Hyperparameter



In [ ]:
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.datasets import load_iris

iris = load_iris()
X, y = training_values, training_target

param_grid = {
    'max_depth': [None, 2, 4, 5,8,10],
    'n_estimators': [100, 150, 200, 300],
    'min_samples_split': [2, 4, 5,8,10]
}

K_Fold= KFold(n_splits=5, shuffle=True, random_state=50)
GS = GridSearchCV(estimator=rf_regressor, param_grid=param_grid,scoring='r2', cv= K_Fold)

GS.fit(X, y)

BestParam = GS.best_params_
print("Best Parameters:", BestParam)

BestScore= GS.best_score_
print("Best Score:", BestScore)
